## Finding the most profitable towns in a CCAA given a max price

In [1]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
from collections import Counter

#### Importing geo data from csv file

This enables us to query for a list of towns given an autonomous community

In [2]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

#### Get input from user

In [3]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

ccaa:  cataluña


In [4]:
num_properties_per_town = ""
while num_properties_per_town == "":
    try:
        num_properties_per_town = int(input("Minimum number of properties per town: "))
    except: 
        num_properties_per_town = ""

Minimum number of properties per town:  0


In [5]:
filtered_df = geo_data[(geo_data['ccaa'] == ccaa) & (geo_data['n_properties'] > num_properties_per_town)]

In [6]:
max_price = ""
while max_price == "":
    try:
        max_price = int(input("maximum price: "))
    except: 
        max_price = ""

maximum price:  100000


#### List of towns that we need to extract data from

In [7]:
towns = filtered_df[filtered_df['ccaa'] == ccaa].town.to_list()

#### Extracting data from each town

In [8]:
property = "viviendas"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
maximum_price = "pmax="+str(max_price)
metros = "m2="+"50"

In [9]:
#url = "https://www.habitaclia.com/"+property+ascensor+"-"+municipio+".htm?"+habitaciones+baños+maximum_price

In [10]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []
lift = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x], 'lift': lift[:x]}
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(1,3))
            url = "https://www.habitaclia.com/"+property+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []
        lift = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):
            #print(i)
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = ""
                    
                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)
                    lift.append(lift_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''
        
        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town], 'lift': lift[:x_town]}
        df_town = pd.DataFrame.from_dict(dct_town)
        df = df.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        print("no properties found at: ", url)

no properties found at:  https://www.habitaclia.com/viviendas-alella.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-begues.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-bellaterra.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-bruc.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-brull.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-cabrera_de_mar.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-cabrils.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-calders.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-caldes_d_estrac.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-calldetenes.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-calonge_de_segarra.htm?pmax=100000
no properties found at:  htt

#### Output from the web scrapping

In [11]:
df.shape

(9584, 16)

In [12]:
df.head(3)

,name,town,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift
0,"Piso en Plaça sa boada, 35. Oportunidad en e...",Arenys de Mar,Arenys de Mar,Centre,arenys_de_mar,50.0,3.0,1.0,1.898,94900.0,100,yes,[],Tenemos en exclusiva esta vivienda visitas con...,https://www.habitaclia.com/comprar-piso-oportu...,
1,Apartamento Carrer avall (d´). Apartamento en...,Arenys de Mar,Arenys de Mar,Centre,arenys_de_mar,41.0,1.0,1.0,2.171,89000.0,0,no,[5],Disseny i habitatge les presenta este fantásti...,https://www.habitaclia.com/comprar-apartamento...,
2,Apartamento en Zona alta. Apartament /mirador,Arenys de Mar,Arenys de Mar,Zona alta,arenys_de_mar,40.0,1.0,1.0,2.500,100000.0,10000,no,[10],REF PV0743 FIRHABIT Tenemos un apartamento / ...,https://www.habitaclia.com/comprar-apartamento...,


#### Cleaning the df

In [13]:
df1 = df.copy()

In [14]:
df1 = df1.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1 = df1.drop(["town_x", "town_y", "n_properties"], axis=1)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [15]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1['last_update']):
        try:
            df1['last_update'][i] = df1['last_update'][i][0]
        except:
            df1['last_update'][i] = '0'

In [16]:
anytype_to_numerical(df1, columns = numericals)
clean_last_update()

<ipython-input-15-e2688416b0bd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = '0'
<ipython-input-15-e2688416b0bd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = df1['last_update'][i][0]


In [17]:
df1= df1[df1['price']<int(max_price)]

In [18]:
df1['lift'] = None

In [33]:
df1.shape

(6681, 17)

#### Getting info about lifts

In [42]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
neighborhood = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []
lift = []

x = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'neighborhood': neighborhood[:x], 'geo_town':geo_town[:x],'m2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x], 'lift':lift[:x]}
df_lift = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+"ascensor"+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(1,3))
            url = "https://www.habitaclia.com/"+property+"-"+"ascensor"+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        neighborhood = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):
            #print(i)
            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            neighborhood_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""
            lift_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    neighborhood_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[1].strip()
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')
                    lift_temp = 'yes'

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    neighborhood.append(neighborhood_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)
                    lift.append(lift_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''
        
        x_town = min(len(name), len(town), len(area), len(neighborhood), len(geo_town), len(description), len(price), len(last_update), len(url), len(lift))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'neighborhood': neighborhood[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town], 'lift':lift[:x_town]}
        df_town = pd.DataFrame.from_dict(dct_town)
        df_lift = df_lift.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]
    except:
        print("no properties found at: ", url)

no properties found at:  https://www.habitaclia.com/viviendas-ascensor-aguilar_de_segarra.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-aiguafreda.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-alella.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-alpens.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-ametlla_del_valles.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-argensola.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-argentona.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-avia.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-avinyonet_del_penedes.htm?pmax=100000
no properties found at:  https://www.habitaclia.com/viviendas-ascensor-begues.htm?pmax=100000
no properties

In [53]:
df1_lift = df_lift.copy()

In [54]:
df_lift.shape

(3378, 16)

In [55]:
df1_lift.shape

(3378, 16)

In [56]:
df1_lift = df1_lift.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1_lift = df1_lift.drop(["town_x", "town_y", "n_properties"], axis=1)
df1_lift = df1_lift.loc[:, ~df1_lift.columns.str.contains('^Unnamed')]

In [57]:
df1_lift.shape

(3378, 17)

In [58]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']

def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)

def clean_last_update():
    for i,n in enumerate(df1_lift['last_update']):
        try:
            df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
        except:
            df1_lift['last_update'][i] = '0'

In [59]:
anytype_to_numerical(df1_lift, columns = numericals)
clean_last_update()

<ipython-input-58-a281638fdd74>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_lift['last_update'][i] = df1_lift['last_update'][i][0]
<ipython-input-58-a281638fdd74>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_lift['last_update'][i] = '0'


In [60]:
df1_lift.shape

(3378, 17)

In [61]:
df1_lift= df1_lift[df1_lift['price']<int(max_price)]

In [62]:
df1_lift.shape

(2072, 17)

In [63]:
df1_lift.sort_values(by = 'price', ascending = True)

,name,area,neighborhood,geo_town,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,lift,province,ccaa
957,Piso Carrer de bosch i cardellach. Centre / c...,Sabadell,Centre,sabadell,88,2,1,9,790,0,no,0,alquilo piso completamente amueblado en la zon...,https://www.habitaclia.com/fv127146723,yes,barcelona,cataluña
2923,Piso Carrer reding. Eixample / carrer reding,Tarragona,Eixample,tarragona,99,3,2,8,800,0,no,0,Fantástico piso en el centro de Tarragona junt...,https://www.habitaclia.com/fv132503512,yes,tarragona,cataluña
3000,Piso en Ferreries. Piso de 60 m² con 3 habit...,Tortosa,Ferreries,tortosa,74,3,1,338,25000,0,no,0,FINANCIACION AL 100% PISO A LA VENTA CON 3 HAB...,https://www.habitaclia.com/comprar-piso-de_60_...,yes,tarragona,cataluña
3081,Piso en Ferreries. Cuarto con 2 habitaciones...,Tortosa,Ferreries,tortosa,80,2,1,339,27100,2700,no,3,La vivienda mide unos 80.08 metros cuadrados c...,https://www.habitaclia.com/comprar-piso-cuarto...,yes,tarragona,cataluña
2811,Piso Carrer antoni rovira i virgili (d´). Sol...,Tarragona,Nou Eixample Nord,tarragona,102,4,2,294,30000,0,no,0,SOLO INVERSORES. SE VENDE LA NUDA PROPIEDAD Pr...,https://www.habitaclia.com/comprar-piso-solo_i...,yes,tarragona,cataluña
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,Piso en Platja Capellans. Se vende piso 1 ha...,Salou,Platja Capellans,salou,72,1,1,1,99932,0,no,4,No pierdas la oportunidad de ver este piso en ...,https://www.habitaclia.com/comprar-piso-se_ven...,yes,tarragona,cataluña
3336,Piso en Vila-seca Poble. Vila-seca calle onz...,Vila,seca - Vila-seca Poble,vila_seca,78,3,1,1,99990,0,no,16,"Fantástico piso de 3 habitaciones, baño y coci...",https://www.habitaclia.com/comprar-piso-calle_...,yes,tarragona,cataluña
3130,Piso en El Tancat-Mas d´en Gual. Oportunidad...,Vendrell (El),El Tancat-Mas d´en Gual,vendrell,94,4,1,1,99990,17910,no,0,"Gran piso con 4 dormitorios, baño, cocina, sal...",https://www.habitaclia.com/comprar-piso-oportu...,yes,tarragona,cataluña
3106,"Piso en Centre. Piso con 2 dormitorios, 1 ba...",Vendrell (El),Centre,vendrell,59,2,1,1,99990,0,no,0,Piso en venta en el barrio de La Muntanyeta de...,https://www.habitaclia.com/comprar-piso-con_2_...,yes,tarragona,cataluña


In [64]:
list_names_with_lift = list(df1_lift['name'])

In [65]:
final_df = df1.combine_first(df1_lift)

In [66]:
final_df.shape

(7492, 17)

#### Saving the scraped df

In [74]:
today = datetime.now().strftime('%Y.%m.%d')
final_df.to_csv(path_or_buf = '/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/sale/'+today+'_'+ccaa+'_'+str(max_price)+'_'+str(num_properties_per_town)+'.csv')